In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
df = pd.read_csv("/kaggle/input/customer-personality-analysis/marketing_campaign.csv", sep="\t")

In [5]:
df.head()

In [6]:
df.isnull().sum()

In [7]:
df=df.dropna()


In [8]:
df['Education'].unique()

In [9]:
df['Marital_Status'].unique()

In [11]:
# calculating the age of customers 
df['c_year'] = 2021
df['age'] = df['c_year']-df['Year_Birth']

# calculating the no of days as customers
df['c_date'] = '01-01-2015'
df['Dt_Customer'] =pd.to_datetime(df.Dt_Customer)
df['c_date'] = pd.to_datetime(df.c_date)
df['days_customer'] = (df['c_date']-df['Dt_Customer']).dt.days

# reducing some dimensions 

# defining two categories of expenses as on food ( fish+meat+ fruit) and as leisure_Expense ( wine, sweet, Gold)
df['leisure_Expense'] = df['MntWines']+ df['MntSweetProducts']+ df['MntGoldProds']
df['food'] = df['MntFishProducts'] + df['MntFruits'] + df['MntMeatProducts']

# defining accepted_any_cmp if customer have ever taken any campaign offer ( 1- yes, 0-no)
df['accepted_any_cmp']= df['AcceptedCmp1']+df['AcceptedCmp2']+df['AcceptedCmp3']+df['AcceptedCmp4']+df['AcceptedCmp5']+df['Response']
df['accepted_any_cmp']= np.where(df['accepted_any_cmp'] > 0, 1, 0)

# defining martial status as ( 0- single adult, 1- two adults)
mapping = {'Single' : 0, 'Together': 1, 'Married': 1, 'Divorced': 0, 'Widow': 0, 'Alone': 0,
       'Absurd': 0, 'YOLO': 0, 'Graduation': 1, 'PhD': 2, 'Master': 2, 'Basic': 0, '2n Cycle': 2}
df=df.replace({'Marital_Status': mapping, 'Education': mapping}) 

df['kid_teen'] = df['Kidhome'] + df['Teenhome']
df['kid_teen']= np.where(df['kid_teen'] > 0, 1, 0)

df.columns

In [14]:
df=df[['Education', 'Marital_Status', 'Income', 'kid_teen',
       'Recency','leisure_Expense', 'food', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'accepted_any_cmp', 
       'Complain', 'age', 'days_customer']]

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(df)

In [16]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X = pca.fit_transform(X)

In [18]:
X, y = df.iloc[:,:-1],data.iloc[:,-1]

In [20]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [23]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [24]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

In [25]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [26]:
import shap
shap_values = shap.TreeExplainer(xg_reg).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")